In [ ]:
import os
import sys
import nibabel as nib
from nibabel.testing import data_path
import matplotlib.pyplot as plt
from skimage.util.shape import view_as_blocks
from patchify import patchify, unpatchify
import numpy as np
import skimage
import array
from deepbrain import Extractor
import cv2
import xlsxwriter

In [ ]:
#reading mri and ground truth images
file = 'c0002s0027t01'; # the name of the input MRI exam 

# path for brain mri
path_mr = 'Brian Stroke Dataset/' + file + '.nii';

# path for ground truth of brain mri
path_lesion_mask = 'Brian Stroke Dataset/'+ file + '_LesionSmooth.nii'

# Get nibabel image objects
brain_nii = nib.load(path_mr) ;
data_brain = brain_nii.get_fdata()

mask_nii = nib.load(path_lesion_mask);
data_mask = mask_nii.get_fdata();


# Convert to numpy ndarray (dtype: uint16)
brain = np.array(data_brain)
mask = np.array(data_mask)

# check the shape of the brain volume and mask volume have the same size or not 
if brain.shape == mask.shape: 
    print('The brin and mask have the same shape')   
    
    

In [ ]:
def write_meta_header(filename, meta_dict):
    header = ''
    # do not use tags = meta_dict.keys() because the order of tags matters
    tags = ['ObjectType','NDims','BinaryData',
       'BinaryDataByteOrderMSB','CompressedData','CompressedDataSize',
       'TransformMatrix','Offset','CenterOfRotation',
       'AnatomicalOrientation',
       'ElementSpacing',
       'DimSize',
       'ElementType',
       'ElementDataFile',
       'Comment','SeriesDescription','AcquisitionDate','AcquisitionTime','StudyDate','StudyTime']
    for tag in tags:
        if tag in meta_dict.keys():
            header += '%s = %s\n'%(tag,meta_dict[tag])
    f = open(filename,'w')
    f.write(header)
    f.close()
    
def dump_raw_data(filename, data):
    """ Write the data into a raw format file. Big endian is always used. """
    #Begin 3D fix
    data=data.reshape([data.shape[0],data.shape[1]*data.shape[2]])
    #End 3D fix
    rawfile = open(filename,'wb')
    a = array.array('f')
    for o in data:
        a.fromlist(list(o))
    #if is_little_endian():
    #    a.byteswap()
    a.tofile(rawfile)
    rawfile.close()
    
def write_mhd_file(mhdfile, data, dsize):
    assert(mhdfile[-4:]=='.mhd')
    meta_dict = {}
    meta_dict['ObjectType'] = 'Image'
    meta_dict['BinaryData'] = 'True'
    meta_dict['BinaryDataByteOrderMSB'] = 'False'
    meta_dict['ElementType'] = 'MET_FLOAT'
    meta_dict['NDims'] = str(len(dsize))
    meta_dict['DimSize'] = ' '.join([str(i) for i in dsize])
    meta_dict['ElementDataFile'] = os.path.split(mhdfile)[1].replace('.mhd','.raw')
    write_meta_header(mhdfile, meta_dict)

    pwd = os.path.split(mhdfile)[0]
    if pwd:
        data_file = pwd +'/' + meta_dict['ElementDataFile']
    else:
        data_file = meta_dict['ElementDataFile']

    dump_raw_data(data_file, data)    
    
def save_lesion(location, patch):
    np.save(str(location) + '.npy', patch)
    write_mhd_file(str(location) + '.mhd', patch, patch.shape[::-1])

In [ ]:
# Skull stripping using deepBrain (credit: https://pypi.org/project/deepbrain/)
ext = Extractor()

# `prob` will be a 3d numpy image containing probability of being brain tissue for each of the voxels in `img`
prob = ext.run(brain) 

# mask can be obtained as:
skull_mask = prob > 0.5
idx = (skull_mask==0)
skull = brain
skull[idx] = skull_mask[idx];


# save the extracted skull 
#skull_location = 'C:/Users/DELL/Desktop/Skulls/'+ file 
#save_lesion(skull_location, skull)     


# normalize extracted skull by subtracting mean and dividing std 
# mean_skull =  skull.mean()
# std_skull = skull.std()
# normalized_skull  = (skull-mean_skull)/std_skull

In [ ]:
def get_data(data_dir, anns_dir):
    length = len(data_dir)
    data_out = []
    for i in range(length):
        patch = np.load(data_dir[i])
        label = np.load(anns_dir[i])
        data_out.append((patch, label))
    return data_out

In [ ]:
# perform varitial mode decomposition (VMD) 
from vmdpy import VMD    #(https://pypi.org/project/vmdpy/)
vmd_skull = []
def decompose_VMD(skull_stripped_volume)
    num_slices = skull_stripped_volume.shape[2]
    for i in range(num_slices):
        alpha = 1000  # moderate bandwidth constraint 
        tau = 0.5     # noise-tolerance (no strict fidelity enforcement)  
        K = 5         # number of mode
        DC = 1        # no DC part imposed  
        init = 1      # initialize omegas uniformly 
        tol = K*1e-6
        #. Run VMD 
        f= lesion_slice
        u, u_hat, omega = VMD(f, alpha, tau, K, DC, init, tol)  
        vmd_slice = u_tmp[3,:,:] # choose mode three
        vmd_skull = [:,:,vmd_slice]
    return vmd_skull


In [ ]:
# create overlapping patches from vmd skull
height = vmd_skull.shape[0]
width = vmd_skull.shape[1]
depth = vmd_skull.shape[2]
print('Image Height       : ',height)
print('Image Width        : ',width)
print('Number of Slices : ',depth)


# before dividing to patches, pre-process the 3D image to get the equal patches 

patch_size = 64;

patches_y,remainder_y = divmod(height,patch_size);
patches_x,remainder_x = divmod(width,patch_size);
patches_z,remainder_z = divmod(depth,patch_size);
print('Number of patchces in y axis : ', patches_y)
print('Number of patchces in x axis : ', patches_x)
print('Number of patchces in z axis : ', patches_z)


# after knowing the remainder in each dimension we try padding 
# if remainder_y or remainder_x or remainder_z is greater than zero
# then perform padding 

if remainder_y!= 0:
    padding_size_y = patch_size-remainder_y
else:
    padding_size_y = 0
    
if remainder_x!= 0:
    padding_size_x = patch_size-remainder_x
else:
    padding_size_x = 0
    
if remainder_z!= 0:
    padding_size_z = patch_size-remainder_z
else:
    padding_size_z = 0


if remainder_y!= 0 or remainder_x != 0 or remainder_z != 0:    
    print('Padding is required')
    padded_skull = np.pad(vmd_skull,((0,padding_size_y),(0,padding_size_x),(0,padding_size_z)),'constant')
    padded_mask = np.pad(mask,((0,padding_size_y),(0,padding_size_x),(0,padding_size_z)),'constant')
    
padded_skull.shape

In [ ]:
# dividing patches by overlapping 10 voxels
from numpy.lib import stride_tricks
def cutup(data, blck, strd):
    sh = np.array(data.shape)
    blck = np.asanyarray(blck)
    strd = np.asanyarray(strd)
    nbl = (sh - blck) // strd + 1
    strides = np.r_[data.strides * strd, data.strides]
    dims = np.r_[nbl, blck]
    data6 = stride_tricks.as_strided(data, strides=strides, shape=dims)
    return data6#.reshape(-1, *blck)

patches_skull = cutup(padded_skull,(patch_size,patch_size,patch_size), (54,54,54))
patches_mask = cutup(padded_mask,(patch_size,patch_size,patch_size), (54,54,54))

patches_brain_dimension = patches_skull.shape
patches_mask_dimension = patches_mask.shape
print('Dimension patches by patchify : ',patches_mask_dimension)

In [ ]:
num_patches_y = patches_brain_dimension[0]
num_patches_x = patches_brain_dimension[1]
num_patches_z = patches_brain_dimension[2]

for row in range(num_patches_y): 
    for col in range(num_patches_x): 
        for dep in range(num_patches_z):

            patch_index_str = str(row) * str(col) * str(dep)

            current_patch_skull =  patches_skull[row][col][dep]  # reference number of the patch (eg- patch_id_1 = [row=1,col=1,depth=1])
            current_patch_label = patches_mask[row][col][dep]    # reference number of the associated mask
            #current_patch_label_overlay = current_patch_skull*current_patch_label


            # patch trimming (if the current patch is only zero voxels, we will trim it out )
            sum_ele_brain = np.sum(current_patch_skull) # sum of all elements in the current skull patch
            sum_ele_mask = np.sum(current_patch_label) # sum of all elements in current patch label  


            if (sum_ele_brain > 0 and sum_ele_mask > 0):
                location_lesion = 'pre_processed dataset/Patch/'+ file + '_' +  patch_index_str
                location_label = 'pre_processed dataset/Label/'+ file +'_' +  patch_index_str
                #location_label_overlay = 'E:/Brin Stroke Detection/Experiment/Dataset/Stoke/LabelOverlay/'+ file +'_lable_patch_' +  patch_index_str 
                status = 'Stroke'
                save_lesion(location_lesion, current_patch_skull)
                save_lesion(location_label, current_patch_label)
                #save_lesion(location_label_overlay, current_patch_label_overlay)
            elif (sum_ele_brain > 0 and sum_ele_mask == 0):
                location_lesion = 'pre_processed dataset/Patch/'+ file +'_' +  patch_index_str
                location_label = 'pre_processed dataset/Label/'+ file +'_' +  patch_index_str 
                status ='Non-Stroke'
                save_lesion(location_lesion, current_patch_skull)
                save_lesion(location_label, current_patch_label)

            else:
                status = "The patch is trimed out"